In [5]:
!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
!pip install --upgrade google-cloud-bigquery

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Data-mining-afa11af25388.json"
import bq_helper
import pandas
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
noaa = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="ghcn_d")

In [4]:
query1 = """SELECT
  id,
  name,
  state,
  latitude,
  longitude
FROM
  `bigquery-public-data.ghcn_d.ghcnd_stations`
WHERE
  latitude > 41.7
  AND latitude < 42
  AND longitude > -87.7
  AND longitude < -87.5;
        """
response1 = noaa.query_to_pandas_safe(query1)
response1.head(10)

,id,name,state,latitude,longitude
0,USC00111562,CHICAGO SAN DIST OFC,IL,41.9000,-87.6333
1,US1ILCK0117,CHICAGO 3.0 NW,IL,41.9125,-87.6695
2,US1ILCK0108,BEVERLY 0.3 N,IL,41.7165,-87.6710
3,US1ILCK0104,LINCOLNWOOD 5.1 SE,IL,41.9494,-87.6703
4,US1ILCK0151,EVERGREEN PARK 0.5 NE,IL,41.7274,-87.6949
5,US1ILCK0122,CHICAGO 6.5 NNE,IL,41.9272,-87.6517
6,USC00111557,CHICAGO RACINE PUMP,IL,41.8167,-87.6500
7,US1ILCK0097,CHICAGO 6.8 NNE,IL,41.9301,-87.6393
8,USC00111584,CHICAGO WB CITY 2,IL,41.8833,-87.6333
9,US1ILCK0010,CHICAGO 3.0 N,IL,41.8798,-87.6823


In [6]:
query2 = """SELECT
  wx.date,
  wx.value/10.0 AS prcp
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2015` AS wx
WHERE
  id = 'USW00094846'
  AND qflag IS NULL
  AND element = 'PRCP'
ORDER BY wx.date;
        """
response2 = noaa.query_to_pandas_safe(query2, max_gb_scanned=10)
response2.head(10)

,date,prcp
0,2015-01-01,0.0
1,2015-01-02,0.0
2,2015-01-03,14.5
3,2015-01-04,3.3
4,2015-01-05,3.8
5,2015-01-06,0.3
6,2015-01-07,0.0
7,2015-01-08,1.5
8,2015-01-09,0.0
9,2015-01-10,0.0


In [5]:
query3 = """SELECT
  stations.id,
  weather.date,
  weather.element,
  weather.value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2015` AS weather INNER JOIN `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations ON 
  weather.id = stations.id
WHERE
  weather.element = 'TOBS'
  AND stations.latitude > 41.7
  AND stations.latitude < 42
  AND stations.longitude > -87.7
  AND stations.longitude < -87.5;
        """
response2 = noaa.query_to_pandas_safe(query3, max_gb_scanned=10)
response2.head(10)

,id,date,element,value
0,USC00111550,2015-07-22,TOBS,217.0
1,USC00111550,2015-02-16,TOBS,-89.0
2,USC00111550,2015-09-07,TOBS,250.0
3,USC00111550,2015-02-22,TOBS,-139.0
4,USC00111550,2015-07-19,TOBS,211.0
5,USC00111550,2015-02-25,TOBS,-61.0
6,USC00111550,2015-10-30,TOBS,94.0
7,USC00111550,2015-06-05,TOBS,106.0
8,USC00111550,2015-10-01,TOBS,122.0
9,USC00111550,2015-10-08,TOBS,189.0
